# Machine Learning and its application in Muon Tomography

Machine Learning inspired techniques can be applied to Muon Tomography to resolve many of associated problems. The two problems of interest to us are *low resolution tomograms* and *data inefficiency*. In this notebook, I am discussing the use of ML as applied to MT to solve problem of data inefficiency. 

## Data Inefficiency Problem

![pic](mt.jpeg)

We define "good muon events" as events with 4 by 4 coincidence hits. Such events have defined spatial points in both top and bottom trays. For example, for the hit on the right the information
$$ A (x_1,y_1) $$
$$ B (x_2,y_2) $$

are known. Due to limited capabilities of our instruments, such events are rare (less than $40\%$ withour best set up) with many events following the trend of the hit on the left. 

$$ C (x_3,y_3) $$
$$ D (x_4,?) $$

We know that $y_4 \in (y_{min}, y_{max})$ and is discernable by $y_{res}$ which is the limited hardware spatial resolution of the telescope. Now, the question is what this $y_4$ may be? 

Once we know how to address this question, we can reduce our data redundancy drastically and improve our overall resolution and efficiency of our schema.

## Track reconstruction using non-perfect events

Each event is assigned a probability that describes the likelihood of using information from that event to reconstruct the muon track. Such a scheme means that events with 4 by 4 coincidence hits have a probability of 1 and events with no hits have probability 0. Events with 3 by 4 hits are thus also assigned a probability.

The way we assign this probability requires the use of some neural networks - *RNN's and LSTM's*. The idea is as follows:

1. Contextualize the entire dataset into a RNN framework
2. Calculate key statistics of overall dataset and individual events
3. Locate events with missing information (i.e. not perfect 4 by 4 coincidence hits)
4. Assign probabilities to candidates for the missing information using wholistic statistics, TDC data, and the angular distribution of muons.
4. Use RNN's and LSTM's to predict the missing datum in 3 by 4 events using the probabilistic approach.


## Probability distribution of the muon hit of a single spatial dimension

Using the coordinate with full information *(C in our case)* and the other spatial coordinate ($x_4$), we assign probabilites to each $y_i \in (y_{min}, y_{max})$. Formally, it can be represented using Bayes' theorem.

$$ P(y_4 = y_i | x_3,y_3,x_4)  $$

Now, the question is how do we create such a probability distribution? What controlled/measured factors can help us make such decisions?

### No information known

When we know previous/extra information about the muon hit, one can imagine the distribution is uniform on the space of the spatial dimension.

$$ P(y_4 = y_i | x_3,y_3,x_4) \sim \mathcal{U} (y_{min}, y_{max})  $$

### One spatial coordinate and one single spatial dimension known

The distribution for such a case is much more complicated. The following are some ways of generating the distribution.

#### Exact Aggregate Data Approach

In this approach, we simply assign the probabilities based on the most likely value from the set of "good" events that share the exact coordinates with the event under consideration. The probability simply becomes a fraction of the two aggregate type of events.

 $$ f_1(y_i) =  P(y_4 = y_i) = \frac{E(y_i,x_3,y_3,x_4)}{E(x_3,y_3,x_4)} $$
    
    
#### Similar Aggregate Data Approach
    
We do a similar thing like the **Exact Aggregate Data Approach** with the added consideration of neighbors to influence the statistic. For a tolerance unit of $\chi$. The following is the probability distribution.

 $$f_2(y_i) = P(y_4 = y_i) = \frac{E(y_i,x_3+\chi,y_3+\chi,x_4+\chi)}{E(x_3+\chi,y_3+\chi,x_4+\chi)} $$
   
   
#### Shift of TDC Approach  

Since, we have demonstrated that TDC values are correlated with the transverse distance of the muon hit along the scintilator bar. It is possible to extrapolate 2D information from such values. In this approach, we do just that to determine the missing spatial dimension.

Let's consider the following case of event D.

**Y and X direction has been mistakenly reveresed here in the plots**

![pic2](sec.png)

We are classing the groups of muon hits along the readout channel of event D into 4 arbitrary groups - W, Q,Y,T. We know that there is a relationship between the average TDC measured at the one of the channels of this group and the measured distance along the axis as illustrated by the following figure. 

![pic3](demo1.png)

The discernable peaks/mean here represent the different "classes". The vector of such statistic is directly correlated with the value of the missing orthogonal dimension. Using this relationship - $y_p = m\mu + c$-, we can predict the missing dimension ($y_4$ in our case).

![pic4](demo2.png)

We define a function $g(y_4 = y_1)$ such that

$$g(y_4 = y_i) = \frac{\lvert (m\mu_{known} + c)-y_i \lvert}{y_{min}+y_{max}}$$

Thus,

 $$f_3(y_i) = P(y_4 = y_i | \mu = \mu_{known}) =  \left\{
\begin{array}{ll}
      1 - g(y_4 = y_i)  & g(y_4 = y_i) \le 1 \\
      0 & g(y_4 = y_i) \ge 1 \\
\end{array} 
\right. $$




#### Most Common Zenith Angle Approach

In this approach, we select the missing spatial dimension such that the angle resembles the most common zenith angle, $\alpha_{mean}$ with $\vec \alpha$ being the vector of angles computed from the data set.

$$ \alpha_{mean} = E(\vec \alpha) $$

Since the space of $y_i$ is discrete. It is, thus, possible to generate a zenith angle distribution over such a space given the set $x_3,y_3,x_4$ is known as is the case in the event of interest.

For example, lets consider such an arbitrary distribution.

<img src="plot2.png" alt="Example" width="600" height="800">

We define a function $d(y_i)$ (given $x_3,y_3,x_4$) as follows:

$$ d(y_i | x_3,y_3,x_4) = \frac{\lvert \alpha_{mean} - \alpha_i(y_i) \lvert}{\alpha_{mean}}  $$

Thus, using this function we generate a probability distribution that selects for the most common zenith angle.

 $$f_4(y_i) = P(y_4 = y_i) = 1 - d(y_i | x_3,y_3,x_4)  $$

The best method should be some combination of all such approaches. This is where ML comes in. The ML scheme would work to solve for the coffecients that of such a weighted sum to **maximize** resolution of tomogram and **minimize** data inefficiency.

## Applying ML

### Comprehensive Probability Function
$$ P(y_4 = y_i) = Af_1(y_i) + Bf_2(y_i) + Cf_3(y_i) + Df_4(y_i)$$

Let $\vec \theta$ be the vector of the coefficients.

$$\vec \theta = \begin{bmatrix}
    A \\
    B \\
    C \\
    D
\end{bmatrix}$$

### Objective Function for Training

Here, $E_\text{total}$ is the total number of events in the data set and $E_\text{used}(\theta)$ is the number of events that are being used for analysis purposes as dictated by $\theta$.

$$\underset{\boldsymbol{\theta}}
{{\text{minimize}}}\;\; E_\text{total} - E_\text{used}(\theta)$$

# Implementation of the Algorithm

For the implementation of the algorithm discussed I will be using runs **56 (with bricks)** and **63 (no bricks)**. The reason behind this selection is to compare the improvement *(if any)* as a result of this new algorithm.

In [1]:
# import the files
from MuonDataFrame import *
file1 = "processed_data/events_data_frame_56.h5"
file2 = "processed_data/events_data_frame_63.h5"

# create the MDFO's

## with bricks
mdfob = MuonDataFrame(file1, isNew=False, d1="last") #Muon Data Frame Object 
mdfb = mdfob.events_df

## no bricks
mdfonb = MuonDataFrame(file2, isNew=False, d1="last") #Muon Data Frame Object 
mdfnb = mdfonb.events_df

In [3]:
mdfb.columns

Index(['event_num', 'event_time', 'deadtime', 'ADC', 'TDC', 'l1hit', 'l2hit',
       'l3hit', 'l4hit', 'r1hit', 'r2hit', 'r3hit', 'r4hit', 'SCh0', 'SCh1',
       'SCh2', 'SCh3', 'SCh4', 'SCh5', 'SCh6', 'SCh7', 'SCh8', 'SCh9', 'SCh10',
       'SCh11', 'L1', 'R1', 'L2', 'R2', 'L3', 'R3', 'L4', 'R4', 'TopCounter',
       'BottomCounter', 'sumL1', 'sumL2', 'sumL3', 'sumL4', 'diffL1', 'diffL2',
       'diffL3', 'diffL4', 'asymL1', 'asymL2', 'asymL3', 'asymL4', 'numLHit',
       'theta_x1', 'theta_y1', 'theta_x2', 'theta_y2', 'z_angle'],
      dtype='object')

We need to identify events with $3/4$ hits.

In [31]:
mdfb3 = mdfb.loc[mdfb['numLHit'] == 7]
mdfnb3 = mdfnb.loc[mdfnb['numLHit'] == 7]

Now, we identify the missing spatial coordinate.

In [72]:
mdfb_cp = mdfb3[['event_num','asymL1','asymL2','asymL3','asymL4']]
mdfnb_cp = mdfnb3[['event_num','asymL1','asymL2','asymL3','asymL4']]

# bricks
mdfb_cp['missing'] = mdfb_cp.isna().dot(mdfb_cp.columns)
# no bricks
mdfnb_cp['missing'] = mdfnb_cp.isna().dot(mdfnb_cp.columns)

In [73]:
mdfb_cp

,event_num,asymL1,asymL2,asymL3,asymL4,missing
0,0,-0.167513,NaN,0.244186,-0.211429,asymL2
1,1,0.038462,-0.004785,-0.294118,NaN,asymL4
5,5,0.109948,0.119617,NaN,-0.082126,asymL3
15,15,-0.187500,NaN,-0.284091,-0.224242,asymL2
18,18,-0.040816,0.253886,0.136364,NaN,asymL4
...,...,...,...,...,...,...
199976,199976,0.255102,0.037037,-0.289941,NaN,asymL4
199978,199978,0.214286,-0.137056,0.040000,NaN,asymL4
199987,199987,0.182320,-0.046154,0.000000,NaN,asymL4
199993,199993,0.252525,0.230047,0.242938,NaN,asymL4


In [74]:
mdfnb_cp

,event_num,asymL1,asymL2,asymL3,asymL4,missing
4,4,NaN,0.033175,0.281768,-0.289617,asymL1
8,8,-0.049505,NaN,0.200000,-0.122807,asymL2
9,9,0.177083,-0.134021,0.000000,NaN,asymL4
11,11,0.041237,0.000000,NaN,0.040936,asymL3
14,14,-0.056410,NaN,0.047059,0.192982,asymL2
...,...,...,...,...,...,...
199986,199986,-0.144385,-0.005236,0.297619,NaN,asymL4
199991,199991,-0.049724,0.043956,-0.164557,NaN,asymL4
199992,199992,0.144385,NaN,0.253012,-0.176471,asymL2
199996,199996,-0.183673,0.179487,-0.043478,NaN,asymL4


In [137]:
#mdfob.getAsymmetry1DPlots()

In [138]:
#mdfb['asymL1'].hist(bins=50,range=(-0.5,0.5))
#plt.show()

In [132]:
"""
def getPD(term,df):
    df[term].plot(kind='kde')
    plt.title(term)
    plt.xlim([-0.5, 0.5])
    plt.show()
    
def getAllPDs(df):
    plt.figure(1)
    
    plt.subplot(221)
    df['asymL1'].plot(kind='kde')
    plt.title('asymL1', fontsize=10)
    plt.xlim([-0.5, 0.5])
    
    plt.subplot(222)
    df['asymL2'].plot(kind='kde')
    plt.xlim([-0.5, 0.5])
    plt.title('asymL2', fontsize=10)
    
    plt.subplot(223)
    df['asymL3'].plot(kind='kde')
    plt.title('asymL3', fontsize=10)
    plt.xlim([-0.5, 0.5])
    
    plt.subplot(224)
    df['asymL4'].plot(kind='kde')
    plt.title('asymL4', fontsize=10)
    plt.xlim([-0.5, 0.5])
    plt.tight_layout()
    plt.show()
"""

"\ndef getPD(term,df):\n    df[term].plot(kind='kde')\n    plt.title(term)\n    plt.xlim([-0.5, 0.5])\n    plt.show()\n    \ndef getAllPDs(df):\n    plt.figure(1)\n    \n    plt.subplot(221)\n    df['asymL1'].plot(kind='kde')\n    plt.title('asymL1', fontsize=10)\n    plt.xlim([-0.5, 0.5])\n    \n    plt.subplot(222)\n    df['asymL2'].plot(kind='kde')\n    plt.xlim([-0.5, 0.5])\n    plt.title('asymL2', fontsize=10)\n    \n    plt.subplot(223)\n    df['asymL3'].plot(kind='kde')\n    plt.title('asymL3', fontsize=10)\n    plt.xlim([-0.5, 0.5])\n    \n    plt.subplot(224)\n    df['asymL4'].plot(kind='kde')\n    plt.title('asymL4', fontsize=10)\n    plt.xlim([-0.5, 0.5])\n    plt.tight_layout()\n    plt.show()\n"

In [133]:
#getAllPDs(mdfb)

In [134]:
#getAllPDs(mdfnb)

In [135]:
"""
mdfob.reload()
mdfob.keepEvents("z_angle",5,"<=")
mdfb4 = mdfob.events_df
"""

'\nmdfob.reload()\nmdfob.keepEvents("z_angle",5,"<=")\nmdfb4 = mdfob.events_df\n'

In [136]:
#getAllPDs(mdfb4)

Since, the asymmetry space is continuous and not discrete we either need to **map the asymmetry space into a discretized physical space** or treat the **asymmetry space as discrete and continue with the algorithm**. We will try out both of these approaches. 

We will first make the assumption that asymmetry space is discrete and follow along with the algorithm.

### Discrete Asymmetry Space

Now, we want to generate 

#### 